In [5]:
import re
import numpy as np
import time

#словарь, хранящий код пословесно. Ключ - название файла
base = {}
#словарь, хранящий строку токенов. Ключ - название файла
sequences = {}
#токены для создания последовательности лексем
tokens = {'for': 'C',
          'while': 'C',
          'break': 'C',
          'continue': 'C', 
          '+': 'O',
          '-': 'O',
          '*': 'O',
          '**': 'O',
          '/': 'O',
          '//': 'O',
          '%': 'O',
          '=': 'O',
          '+=': 'S',
          '-=': 'S',
          '/=': 'S',
          '*=': 'S',
          '==': 'S', 
          'and': 'L',
          'or': 'L',
          'not': 'L',
          'in': 'L',
          'input': 'K',
          'def': 'K',
          'import': 'K',
          'print': 'K',
          'if': 'I',
          'else': 'I',
          'elif': 'I',
          '()': 'F',
          '[': 'F',
          ']': 'F'}

#функция добавления значений в словарь
def base_add(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = key
        dictionary[key] = value
    else:
        dictionary[value][key] += value
    return (dictionary)

#создание словаря для лексем и файлов            
def base_sequences(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = key
        dictionary[key] = value
    else:
        dictionary[key] = value
    return (dictionary)

#вернуть ключ по значению
#def get_key(d, value_search):
    #for key, value in d.items():
        #for i in range(len(value)):
            #if value_search in value:
                #return key

#алгоритм Вагнера-Фишера
def fish(array, stroka_1, stroka_2):
    if stroka_1 == stroka_2:
        return (0)
    numm_1 = 0
    for letter_1 in stroka_1:
        numm_1 += 1
        numm_2 = 0
        for letter_2 in stroka_2:
            numm_2 += 1
            if letter_1 == letter_2:
                array[numm_1, numm_2] = array[numm_1 - 1, numm_2 - 1]
            else:
                array[numm_1, numm_2] = min(array[numm_1 - 1, numm_2 - 1], array[numm_1, numm_2 - 1], array[numm_1 - 1, numm_2]) + 1
    return (1 - (array[-1][-1] / max(numm_1, numm_2))) * 100            

#мелкая копия словаря для удаления файла, с которым идет сравнение
#def minus_key(dictionary, key):
    #shallow_copy = dict(dictionary)
    #del shallow_copy[key]
    #return shallow_copy

#создание матрицы для расчета расстояния Левенштейна
def matrix(source_1, source_2):
    a = np.zeros((len(sequences[source_1]) + 1, len(sequences[source_2]) + 1), int)
    a[0] = range(len(a[1]))
    for i in range(len(sequences[source_1]) + 1):
        a[i, 0] = i
    return a
    
source = ['Mouse1.py', 'Mouse2.py', 'circle.py', 'Movement.py', 'Movement1.py', 'Movement2.py']
#source = ['Mouse.py', 'Mouse1.py', 'Mouse2.py']

start_time = time.time()

#чтение файлов и запись кода пословесно в словарь. Ключ - название файла
for i in range(len(source)): 
    code = [] 
    with open (source[i]) as prog:
        for line in prog:
            #нормализация текста
            line = re.sub("^\s+|\t|\n|\r|\s+$", '', line)
            line = line.lower()
            code += line.split(' ')
        #добавление в словарь base пословно
        base_add(base, source[i], code)

#составление последовательности лексем на основании токенов
for i in range(len(source)):
    for key1 in base.keys():
        stroka = ''
        for word in base[key1]:
            if word in tokens:
                stroka += tokens[word]
                base_sequences(sequences, key1, stroka)

for i in range(len(source)):
    key_1, value_1 = source[i], sequences[source[i]]
    for g in range(i, len(source)):
        if g == i:
            continue
        key_2, value_2 = source[g], sequences[source[g]]
        a = matrix(key_1, key_2)
        print('Сравнивается:', key_1, ' и ', key_2)
        print('Заимствований:', '%2.2f'% fish(a, sequences[key_1], sequences[key_2]), '%')
    
print("-— %s секунд —-" % (time.time() - start_time))

Сравнивается: Mouse1.py  и  Mouse2.py
Заимствований: 73.91 %
Сравнивается: Mouse1.py  и  circle.py
Заимствований: 50.00 %
Сравнивается: Mouse1.py  и  Movement.py
Заимствований: 37.29 %
Сравнивается: Mouse1.py  и  Movement1.py
Заимствований: 53.66 %
Сравнивается: Mouse1.py  и  Movement2.py
Заимствований: 70.37 %
Сравнивается: Mouse2.py  и  circle.py
Заимствований: 47.37 %
Сравнивается: Mouse2.py  и  Movement.py
Заимствований: 32.20 %
Сравнивается: Mouse2.py  и  Movement1.py
Заимствований: 46.34 %
Сравнивается: Mouse2.py  и  Movement2.py
Заимствований: 66.67 %
Сравнивается: circle.py  и  Movement.py
Заимствований: 49.15 %
Сравнивается: circle.py  и  Movement1.py
Заимствований: 56.10 %
Сравнивается: circle.py  и  Movement2.py
Заимствований: 63.16 %
Сравнивается: Movement.py  и  Movement1.py
Заимствований: 69.49 %
Сравнивается: Movement.py  и  Movement2.py
Заимствований: 45.76 %
Сравнивается: Movement1.py  и  Movement2.py
Заимствований: 65.85 %
-— 0.02103400230407715 секунд —-


In [6]:
import pandas as pd

df = pd.DataFrame({
    'File': source
})
df['Sequence'] = df['File']
df['Sequence'] = df['Sequence'].map(sequences)
df

,File,Sequence
0,Mouse1.py,KOOOOOOOOOOOCCLISISISCS
1,Mouse2.py,KOOOOOOOOOOCCLISOOI
2,circle.py,KKOOOOOOOOOOOOOOOOCLSOCCLISCLCLIOOOOOS
3,Movement.py,KOOOOOOOOOOOOOOOOOOOOOOCCLISISISOISOISOISOISIL...
4,Movement1.py,KOOOOOOOOOOOOOOOOCCLISISISOISOISILOISSISS
5,Movement2.py,KOOOOOOOOOOOOOOOOCCLISOISIS


In [7]:
df_res = pd.DataFrame({'File': source[:]})

for i in range(len(source)):
    key_1, value_1 = source[i], sequences[source[i]]
    result = []
    for g in range(len(source)):
        key_2, value_2 = source[g], sequences[source[g]]
        a = matrix(key_1, key_2)
        result.append(fish(a, sequences[key_1], sequences[key_2]))
    df_res[key_1] = result

df_res

,File,Mouse1.py,Mouse2.py,circle.py,Movement.py,Movement1.py,Movement2.py
0,Mouse1.py,0.000000,73.913043,50.000000,37.288136,53.658537,70.370370
1,Mouse2.py,73.913043,0.000000,47.368421,32.203390,46.341463,66.666667
2,circle.py,50.000000,47.368421,0.000000,49.152542,56.097561,63.157895
3,Movement.py,37.288136,32.203390,49.152542,0.000000,69.491525,45.762712
4,Movement1.py,53.658537,46.341463,56.097561,69.491525,0.000000,65.853659
5,Movement2.py,70.370370,66.666667,63.157895,45.762712,65.853659,0.000000
